### We will be using pytorch datasets and data loaders to implement  residual U-net model 

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, sampler, Subset, WeightedRandomSampler


from torchvision import transforms, datasets

import logging
import numpy as np
from utils import *


import time
import copy

import skimage 
from skimage import io 
import logging
from PIL import Image

import os 
from os.path import splitext
from os import listdir

import numpy as np
import pandas as pd
from pandas import DataFrame

from torchvision.transforms import Compose, ToTensor, Resize, ToPILImage
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000  # incase PIL gives error

In [20]:

# VM , glued images are stored at path '/home/abharani/cs231n_project/glued_images/data'
root_dir = '/home/abharani/cs231n_project'
data_dir = '/home/abharani/cs231n_project/data'

# Local 
# root_dir = '/Users/abharani/Documents/myworkspace/cs231n_project'
# data_dir = '/Users/abharani/Documents/myworkspace/cs231n_project/data'


dir_checkpoint = os.path.join(root_dir, 'checkpoints')


### Step1 :  Train , Valid and Test Split 

Afrer saving the images generated from Dataset class ImageFolderWithPaths. Create dataset using the ImageFolder.
Since Data is imbalanced, performing Weighted Random Sampling not Random Sampling.



In [24]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader


def train_val_dataset(dataset, val_split=0.25, generate_small=False):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets_created = {}
    if generate_small:
        print("Generating Small Train, Test Dataset")
        datasets_created['train'] = Subset(dataset, train_idx[0:200])
        datasets_created['test'] = Subset(dataset, val_idx[0:50])
    else:
        datasets_created['train'] = Subset(dataset, train_idx)
        datasets_created['test'] = Subset(dataset, val_idx)        
    return datasets_created

dataset = ImageFolder('/home/abharani/cs231n_project/glued_images/data',
                      transform=Compose([Resize((224,224)),ToTensor(), 
                                        Normalize(mean=[0.485, 0.456, 0.406],
                                                   std=[0.229, 0.224, 0.225])]))


#### Performing Train, Valid and Test Split

In [26]:
dataset_final = {}
datasets_created_trial_1 = train_val_dataset(dataset,val_split=0.10,generate_small=False)
datasets_created_trial_2 = train_val_dataset(datasets_created_trial_1['train'],val_split=0.25,generate_small=False)

dataset_final['test'] = datasets_created_trial_1['test']
dataset_final['train'] = datasets_created_trial_2['train']
dataset_final['val'] = datasets_created_trial_2['test']

print("Dataset set size {}".format(len(dataset)))
print("Train set size {}".format(len(dataset_final['train'])))
print("Validation set size {}".format(len(dataset_final['val'])))
print("Test set size {}".format(len(dataset_final['test'])))

Dataset set size 10615
Train set size 7164
Validation set size 2389
Test set size 1062


In [27]:
# for i in range(len(datasets['train'])):
#     sample = dataset[i]

#     print(i, sample['image'].shape, sample['isup_grade'])

#     if i == 3:
#         plt.show()
#         break

### Step 2: WeightedRandomSampling on Imbalance train set

Performing WeightedRandomSampling using link : https://towardsdatascience.com/pytorch-basics-sampling-samplers-2a0f29f0bf2a

Generate class weights and target_list 

In [28]:
from collections import defaultdict 

count_dict = defaultdict(int)
target_list = []

#Generate target_list of all labels and count dict of all classes

for i, (image, label) in enumerate(dataset_final['train']):    
    count_dict[label] += 1
    target_list.append(label)
    
#     print(i, image.shape,sample)    
#     if i== 3:
#         break


count_dict
print("Distribution of classes: \n", count_dict)

class_count = [i for i in count_dict.values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
print("Class weights : \n", class_weights)

Distribution of classes: 
 defaultdict(<class 'int'>, {2: 912, 4: 831, 0: 1956, 1: 1813, 3: 814, 5: 838})
Class weights : 
 tensor([0.0011, 0.0012, 0.0005, 0.0006, 0.0012, 0.0012])


In [29]:
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

In [30]:
class_weights_all = class_weights[target_list]
class_weights_all

tensor([0.0011, 0.0012, 0.0006,  ..., 0.0012, 0.0005, 0.0006])

In [31]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

### Use gpu and float as dtype

In [32]:

USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print("device set to {}".format(device))
    

base_model = torchvision.models.resnet18(pretrained=False)

base_model = base_model.to(device)

# list(base_model.children())



device set to cuda


### Step 3: Create Model definition

In [33]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class ResNetUNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
        
        self.base_model = torchvision.models.resnet18(pretrained=True)

        self.base_layers = list(base_model.children())                
        
        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)        
        self.layer1_1x1 = convrelu(64, 64, 1, 0)       
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)        
        self.layer2_1x1 = convrelu(128, 128, 1, 0)  
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)        
        self.layer3_1x1 = convrelu(256, 256, 1, 0)  
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)  
        
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)
        
        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)
        
        self.conv_last = nn.Conv2d(64, n_class, 1)
        # MY experiment
        self.fc = nn.Linear(301056, 6)
        
    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)
        
        layer0 = self.layer0(input)            
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)        
        layer4 = self.layer4(layer3)
        
        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)
 
        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)
        
        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)        
        
        out = self.conv_last(x)        

        # MY experiment
        x = out.view(out.size(0), -1)
        x = self.fc(x)
        
        # x = F.adaptive_avg_pool2d(out, output_size=1)
        # print(x.size(),x)

        
        return x



## Use new_train_enhanced_images below for training on images glued ones

### Trying with 8.x image dataset

In [34]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [36]:
num_class = 6

model = ResNetUNet(num_class).to(device)

# freeze backbone layers
# Comment out to finetune further
# for l in model.base_layers:
#     for param in l.parameters():
#         param.requires_grad = False

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)        

# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(dataset_final['train'],batch_size=32, sampler = weighted_sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(dataset_final['val'],batch_size=32, num_workers=4, pin_memory=True)

dataloaders  = {'train': train_loader, 'val': val_loader}

dataset_sizes = {x: len(dataset_final[x]) for x in dataset_final.keys()}



model = train_model(model, criterion, optimizer_ft, scheduler, num_epochs=10)


Epoch 0/9
----------
train Loss: 1.6697 Acc: 0.2994
val Loss: 1.6505 Acc: 0.2842

Epoch 1/9
----------
train Loss: 1.5154 Acc: 0.3836
val Loss: 1.6298 Acc: 0.3361

Epoch 2/9
----------
train Loss: 1.3108 Acc: 0.5021
val Loss: 1.7994 Acc: 0.3010

Epoch 3/9
----------
train Loss: 0.9477 Acc: 0.6830
val Loss: 2.1027 Acc: 0.2901

Epoch 4/9
----------
train Loss: 0.6088 Acc: 0.8131
val Loss: 2.5375 Acc: 0.2813

Epoch 5/9
----------
train Loss: 0.3845 Acc: 0.8911
val Loss: 2.9201 Acc: 0.2784

Epoch 6/9
----------
train Loss: 0.2223 Acc: 0.9443
val Loss: 3.3108 Acc: 0.2825

Epoch 7/9
----------


KeyboardInterrupt: 

### End of Training